In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pandas 数据合并与数据重塑

如果你的数据由多张表组成，那么有时候需要将不同的数据表合并在一起分析,pandas常用的方法分为两类：数据合并和数据重塑。数据合并用于把不同的数据表合并到一个DataFrame中，数据重塑是合并后的DataFrame常拥有多层索引（多维），为了方便理解和计算，需要把多层索引（多维）转变为2维数据。

* 数据合并：合并多张数据表
    * pd.concat:可以沿着一条轴将多个对象堆叠在一起，常用于实现列的合并
    * pd.merge：可根据一个或多个键将不同DataFrame中的行连接起来，类似于数据库的join操作


* 数据重塑：用于重新排列表格型数据
    * df.stack：将数据的列旋转为行
    * df.unstack：将数据的行旋转为列

## 1 pd.concat

* pd.concat([data1, data2], axis=0,join='outer')
    * axis默认为0，表示跨行进行合并，所以是行合并
    * join默认是'outer'外连接，可以选择‘inner’内连接

举例：对股票数据进行分组，然后进行One-Hot编码，在与原数据合并

In [2]:
# 导入数据
stock_data = pd.read_csv('../data/stock_day.csv')
stock_data

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
2018-02-27,23.53,25.88,24.16,23.53,95578.03,0.63,2.68,22.942,22.142,22.875,53782.64,46738.65,55576.11,2.39
2018-02-26,22.80,23.78,23.53,22.80,60985.11,0.69,3.02,22.406,21.955,22.942,40827.52,42736.34,56007.50,1.53
2018-02-23,22.88,23.37,22.82,22.71,52914.01,0.54,2.42,21.938,21.929,23.022,35119.58,41871.97,56372.85,1.32
2018-02-22,22.25,22.76,22.28,22.02,36105.01,0.36,1.64,21.446,21.909,23.137,35397.58,39904.78,60149.60,0.90
2018-02-14,21.49,21.99,21.92,21.48,23331.04,0.44,2.05,21.366,21.923,23.253,33590.21,42935.74,61716.11,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-06,13.17,14.48,14.28,13.13,179831.72,1.12,8.51,13.112,13.112,13.112,115090.18,115090.18,115090.18,6.16
2015-03-05,12.88,13.45,13.16,12.87,93180.39,0.26,2.02,12.820,12.820,12.820,98904.79,98904.79,98904.79,3.19
2015-03-04,12.80,12.92,12.90,12.61,67075.44,0.20,1.57,12.707,12.707,12.707,100812.93,100812.93,100812.93,2.30
2015-03-03,12.52,13.06,12.70,12.52,139071.61,0.18,1.44,12.610,12.610,12.610,117681.67,117681.67,117681.67,4.76


In [3]:
# 筛选出p_change数据
p_change = stock_data['p_change']
p_change

2018-02-27    2.68
2018-02-26    3.02
2018-02-23    2.42
2018-02-22    1.64
2018-02-14    2.05
              ... 
2015-03-06    8.51
2015-03-05    2.02
2015-03-04    1.57
2015-03-03    1.44
2015-03-02    2.62
Name: p_change, Length: 643, dtype: float64

In [4]:
# 我们使用qcut进行分组
q_counts = pd.qcut(p_change,10)
q_counts

2018-02-27    (1.738, 2.938]
2018-02-26     (2.938, 5.27]
2018-02-23    (1.738, 2.938]
2018-02-22     (0.94, 1.738]
2018-02-14    (1.738, 2.938]
                   ...      
2015-03-06     (5.27, 10.03]
2015-03-05    (1.738, 2.938]
2015-03-04     (0.94, 1.738]
2015-03-03     (0.94, 1.738]
2015-03-02    (1.738, 2.938]
Name: p_change, Length: 643, dtype: category
Categories (10, interval[float64]): [(-10.030999999999999, -4.836] < (-4.836, -2.444] < (-2.444, -1.352] < (-1.352, -0.462] ... (0.94, 1.738] < (1.738, 2.938] < (2.938, 5.27] < (5.27, 10.03]]

In [5]:
# 使用get_dummies进行One-Hot编码
dummies = pd.get_dummies(q_counts,prefix='rise')
dummies

,"rise_(-10.030999999999999, -4.836]","rise_(-4.836, -2.444]","rise_(-2.444, -1.352]","rise_(-1.352, -0.462]","rise_(-0.462, 0.26]","rise_(0.26, 0.94]","rise_(0.94, 1.738]","rise_(1.738, 2.938]","rise_(2.938, 5.27]","rise_(5.27, 10.03]"
2018-02-27,0,0,0,0,0,0,0,1,0,0
2018-02-26,0,0,0,0,0,0,0,0,1,0
2018-02-23,0,0,0,0,0,0,0,1,0,0
2018-02-22,0,0,0,0,0,0,1,0,0,0
2018-02-14,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2015-03-06,0,0,0,0,0,0,0,0,0,1
2015-03-05,0,0,0,0,0,0,0,1,0,0
2015-03-04,0,0,0,0,0,0,1,0,0,0
2015-03-03,0,0,0,0,0,0,1,0,0,0


In [6]:
# 使用pd.concat进行数据的合并
pd.concat([stock_data,dummies],axis=1)

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,...,"rise_(-10.030999999999999, -4.836]","rise_(-4.836, -2.444]","rise_(-2.444, -1.352]","rise_(-1.352, -0.462]","rise_(-0.462, 0.26]","rise_(0.26, 0.94]","rise_(0.94, 1.738]","rise_(1.738, 2.938]","rise_(2.938, 5.27]","rise_(5.27, 10.03]"
2018-02-27,23.53,25.88,24.16,23.53,95578.03,0.63,2.68,22.942,22.142,22.875,...,0,0,0,0,0,0,0,1,0,0
2018-02-26,22.80,23.78,23.53,22.80,60985.11,0.69,3.02,22.406,21.955,22.942,...,0,0,0,0,0,0,0,0,1,0
2018-02-23,22.88,23.37,22.82,22.71,52914.01,0.54,2.42,21.938,21.929,23.022,...,0,0,0,0,0,0,0,1,0,0
2018-02-22,22.25,22.76,22.28,22.02,36105.01,0.36,1.64,21.446,21.909,23.137,...,0,0,0,0,0,0,1,0,0,0
2018-02-14,21.49,21.99,21.92,21.48,23331.04,0.44,2.05,21.366,21.923,23.253,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-06,13.17,14.48,14.28,13.13,179831.72,1.12,8.51,13.112,13.112,13.112,...,0,0,0,0,0,0,0,0,0,1
2015-03-05,12.88,13.45,13.16,12.87,93180.39,0.26,2.02,12.820,12.820,12.820,...,0,0,0,0,0,0,0,1,0,0
2015-03-04,12.80,12.92,12.90,12.61,67075.44,0.20,1.57,12.707,12.707,12.707,...,0,0,0,0,0,0,1,0,0,0
2015-03-03,12.52,13.06,12.70,12.52,139071.61,0.18,1.44,12.610,12.610,12.610,...,0,0,0,0,0,0,1,0,0,0


## 2 pd.merge
* pd.merge(left, right, how='inner', on=None)
    * left: DataFrame
    * right: 另一个DataFrame
    * how:按照什么方式连接,['left', 'right', 'outer', 'inner'],默认'inner'
        * 注意：使用外连接，经常会出现缺失值NaN
    * on: 指定的共同键
        * 注意：如果左右两表共同键名字不同，可以选择‘left_on’和‘right_on’参数分别指定左右两表的连接键


下面举例演示pd.merge方法合并数据集：

In [7]:
# 新建左表
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                        'key2': ['K0', 'K1', 'K0', 'K1'],
                        'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']})
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [8]:
# 新建右表
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                        'key2': ['K0', 'K0', 'K0', 'K0'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [9]:
# 使用默认内连接,共同键为'key1'和'key2'
pd.merge(left,right,on=['key1','key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [10]:
# 使用外连接，共同键为'key1'和'key2'
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


## 3 stack

* 将列索引旋转为行索引，完成层级索引
* DataFrame->Series

下面举例说明stack，完成层级索引：

In [11]:
# 新建一个成绩的数据表
df = pd.DataFrame(np.random.randint(0,100, (5,2)), columns=['Chinese', 'English'])
df

,Chinese,English
0,78,75
1,14,91
2,10,74
3,71,45
4,62,35


In [12]:
# 使用stack完成层级索引
stacked = df.stack()
stacked

0  Chinese    78
   English    75
1  Chinese    14
   English    91
2  Chinese    10
   English    74
3  Chinese    71
   English    45
4  Chinese    62
   English    35
dtype: int32

* stacked共有两层索引：
    * 0层索引即是[0,1,2,3,4]那一列
    * 1层索引即是包含['Chinese','English']那一列

## 4 unstack

* 将层级索引展开
* Series->DataFrame
* 默认操作内层索引，即level=-1

In [13]:
# 默认操作内层索引，即level=1或-1
stacked.unstack()

,Chinese,English
0,78,75
1,14,91
2,10,74
3,71,45
4,62,35


In [14]:
# 操作外层索引，即level=0
stacked.unstack(level=0)

,0,1,2,3,4
Chinese,78,14,10,71,62
English,75,91,74,45,35
